In [92]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, VotingClassifier
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.metrics import accuracy_score

In [93]:
data = pd.read_csv("D:/DataSets/Predictive Analysis_ML/Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [94]:
data.drop(columns=["RowNumber","CustomerId","Surname"], inplace=True)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [95]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [96]:
data.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [97]:
data.duplicated().sum()

0

In [98]:
data["Geography"].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [99]:
data["Gender"].value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

In [100]:
data["Exited"].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [101]:
data = pd.get_dummies(data, ["Gender", "Geography"], drop_first=True)
data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Germany,Gender_Spain,Geography_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [102]:
x = data.drop(columns="Exited")
y = data["Exited"]

y.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [103]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2,random_state=None)

In [104]:
y_train.value_counts()

0    6358
1    1642
Name: Exited, dtype: int64

In [105]:
sc = StandardScaler()

x_train_scaled = sc.fit_transform(x_train)
x_test_scaled = sc.transform(x_test)

In [106]:
smote = SMOTE(sampling_strategy="minority")
x_train_sm, y_train_sm = smote.fit_resample(x_train,y_train)

In [107]:
y_train_sm.value_counts()

0    6358
1    6358
Name: Exited, dtype: int64

In [108]:
counter = Counter
print("BEFORE :",counter(y_train))
print("AFTER :",counter(y_train_sm))

BEFORE : Counter({0: 6358, 1: 1642})
AFTER : Counter({0: 6358, 1: 6358})


In [109]:
lor = LogisticRegression()

In [110]:
dt = DecisionTreeClassifier(max_depth=5)

In [111]:
bc = BaggingClassifier()

In [112]:
rf = RandomForestClassifier(max_depth=5)

In [113]:
estimators = [("lr",lor), ("dt",dt), ("rf",rf)]

vc = VotingClassifier(estimators=estimators)

In [114]:
models = [("lor",lor), ("dt",dt), ("vc",vc), ("bc",bc), ("rf",rf)]
for i in models:
    var = i[1]
    var.fit(x_train_sm,y_train_sm)
    pred = var.predict(x_test)
    print("accuracy_score of", i[0], ": ", accuracy_score(y_test, pred)*100)

accuracy_score of lor :  64.1
accuracy_score of dt :  76.0
accuracy_score of vc :  77.14999999999999
accuracy_score of bc :  79.55
accuracy_score of rf :  77.25


In [115]:
score = []
for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=i)
    rf = RandomForestClassifier()
    rf.fit(x_train_sm,y_train_sm)
    pred = rf.predict(x_test)
    score.append(accuracy_score(y_test,pred)*100)
print("i= ",np.argmax(score))
print(score[np.argmax(score)])

i=  1
96.85000000000001


In [118]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=1)
rf = RandomForestClassifier()
rf.fit(x_train_sm,y_train_sm)
pred = rf.predict(x_test)
accuracy_score(y_test,pred)*100

96.35000000000001